### Sample RHESSys workflow

**Note:** This workflow is underdevelopment and still needs documentation

In [1]:
import os
import logging
from rhessys_wf import *
%matplotlib inline

The `RHESSysWorkflow` class is provided as part of the `rhessys_wf` library (imported above) to streamline your interaction with the `RHESSysWorkflows` [core functions](https://github.com/selimnairb/RHESSysWorkflows).  More information about this class can be obtained by executing the `help(RHESSysWorkflow)` command.

Create and instance of the `RHESSysWorkflow` class and assign it to the variable `w`, using the gage id **01589312**, a start date of **2008-01-01**, and an end date of **2010-01-01**.  This command will create a clean directory for the given project name of **mysim** in the JupyterHub's default data directory.

In [2]:
w = RHESSysWorkflow(project_name='mysim', 
                    gageid='01589312',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

Project [mysim] already exists, would you like to remove it [Y/n]? 
Creating a clean directory for project [mysim]
Log file location: /home/jovyan/work/notebooks/data/mysim/mysim.log


The `RHESSysWorkflow` class uses a logging library to document output and errors.  We can display these messages in our notebook by attaching to the logger and redirecting the output to stdout.  While this step is not necessary, it will provide us with more verbose output.

In [3]:
# Get root logger (all other loggers will be derived from this logger's properties)
logger = logging.getLogger()

# assuming only a single handler has been setup (seems to be default in notebook), set that handler to go to stdout.
logger.handlers[0].stream = sys.stdout

Create project from NHD gage id??

In [4]:
w.get_NHDStreamflowGageIdentifiersAndLocation(w.sub_project_folder,w.gageid)
w.get_CatchmentShapefileForNHDStreamflowGage(w.sub_project_folder)
w.get_BoundingboxFromStudyareaShapefile(w.sub_project_folder)
extent = w.get_Extent_from_RHESSysWorkflows_Metadata_File()

INFO:myApp:GetNHDStreamflowGageIdentifiersAndLocation.py -p /home/jovyan/work/notebooks/data/mysim/mysim -g 01589312
INFO:myApp:GetCatchmentShapefileForNHDStreamflowGage.py --overwrite -p /home/jovyan/work/notebooks/data/mysim/mysim
INFO:myApp:GetBoundingboxFromStudyareaShapefile.py -p /home/jovyan/work/notebooks/data/mysim/mysim
INFO:myApp:/home/jovyan/work/notebooks/data/mysim/mysim/metadata.txt
INFO:myApp:
min x = -76.769782
min y = 39.273610
max x = -76.717498
max y = 39.326008



Retrieve ??? from HydroTerre

In [ ]:
w.HydroTerre_RHESSys_ByExtent(extent, w.ht_start_date, w.ht_end_date, w.sub_project_folder)
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = w.sub_project_folder + '/RHESSys_ETV_Data.zip'
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)

do more rhessys prep ...

In [ ]:
output = w.get_USGSDEMForBoundingbox(w.sub_project_folder)
output = w.get_USGSNLCDForDEMExtent(w.sub_project_folder)
output = w.get_SSURGOFeaturesForBoundingbox(w.sub_project_folder)
output =w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)
w.lai_fullpathname_with_ext = w.sub_project_folder + '/RHESSys_ETV/RHESSys_LAI/LAI_Month0.tif'
output = w.Register_LAI_Raster(w.sub_project_folder, w.lai_fullpathname_with_ext, w.publisher)
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, '"RHESSys model for Dead Run 5 watershed near Catonsville, MD"')

import rhessys code...

In [ ]:
output = w.ImportRHESSysSource(w.sub_project_folder)
print 'Finished'

add climate data, delineate watershed....

In [ ]:
output = w.climate_data_fullpathname = w.sub_project_folder + '/RHESSys_ETV/RHESSys_Climate'

output = w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)

output = w.station_data_fullpathname = w.sub_project_folder + '/RHESSys_ETV/RHESSys_Climate'

output = w.DelineateWatershed(w.sub_project_folder, w.dem_cell_threshold, w.areaEstimate)

output = w.GeneratePatchMap(w.sub_project_folder)

generate soil texture ...

In [ ]:
output = w.GenerateSoilTextureMap(w.sub_project_folder, options='--overwrite')

output = w.ImportRasterMapIntoGRASS_LAI(w.sub_project_folder)

output = w.ImportRasterMapIntoGRASS_LANDCOVER(w.sub_project_folder)

generate rhessys simulation inputs???

In [ ]:
output = w.RegisterLandcoverReclassRules(w.sub_project_folder)
output = w.GenerateLandcoverMaps(w.sub_project_folder)
output = w.GenerateWorldTemplate(w.sub_project_folder)
output = w.CreateWorldfile(w.sub_project_folder)
output = w.CreateFlowtable(w.sub_project_folder)
output = w.RunLAIRead(w.sub_project_folder)
output = w.RunCmd(w.sub_project_folder, 3)            

run the rhessys simulation

In [ ]:
output = w.RunModel(w.sub_project_folder)

plot the output

In [ ]:
data =  os.path.join(w.sub_project_folder, 'rhessys/output/test/rhessys_basin.daily')
obs_data =  '/home/jovyan/work/notebooks/data/test_obs.txt'

plot_rhessys_results(
    outfileSuffix = 'test_plot', 
     obs = obs_data, 
     column = 'streamflow',
     data = [data], 
     legend = ['Test simulation'],
     title = 'DR5 streamflow',
     y = 'Streamflow (mm/day)')


plot_rhessys_results('test_plot', 
     obs_data, 
     'streamflow', 
     ['Test simulation'],
        #plottype=PLOT_DEFAULT,
        data=[data], 
        #behavioralData=None,
        #color=['magenta'],
        #linewidth=None,
        #linestyle=None,
        title='DR5 streamflow',
        #x=None,
        y='Streamflow (mm/day)',
        #titlefontsize=12,
        #scatterwidth=1,
        #fontweight='regular',
        #legendfontsize=6,
        #axesfontsize=12,
        #ticklabelfontsize=12,
        #figureX=8,
        #figureY=5,
        #supressObs=False,
        #secondaryData=obs_data,
        #secondaryPlotType='bar',
        #secondaryColumn='precip',
        #secondaryLabel='Rainfall (mm/day)'
                    )

---
## 3. Save the results back into HydroShare

Using the `hs_utils` library, the results of our timeseries analysis can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, and *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Optional*** : define the resource from which this "new" content has been derived.  This is one method for tracking resource provenance.

In [29]:
import hs_utils

# establish a secure connection to HydroShare
hs = hs_utils.hydroshare()

Adding the following system variables:
   HS_USR_NAME = TonyCastronova
   HS_RES_ID = 466d5b3de8a543808d96ded08d861dc5
   HS_RES_TYPE = genericresource
   JUPYTER_HUB_IP = jupyter.uwrl.usu.edu

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => TonyCastronova


In [26]:
# compress the simulation data
!tar -zcf $DATA/mysim.tar.gz $DATA/mysim 

tar: Removing leading `/' from member names


In [30]:
# define HydroShare required metadata
title = 'RHESSys Example'
abstract = 'This is a sample RHESSys simulation'
keywords = ['Jupyterhub', 'RHESSys']

# set the resource type that will be created.
rtype = 'genericresource'

# create a list of files that will be added to the HydroShare resource.
files = [os.path.join(os.environ['DATA'], 'mysim.tar.gz'), # the compressed simulation data
         os.path.join(os.getcwd(), 'rhessys.ipynb')  # this notebook
        ]  

In [33]:
# save the state of the current notebook
from IPython.display import display,Javascript 
display(Javascript('IPython.notebook.save_checkpoint();'))

# create a hydroshare resource containing these data
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          derivedFromId=None,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)

<IPython.core.display.Javascript object>

You have indicated that this resource is NOT derived from any existing HydroShare resource.  Are you sure that this is what you intended? [Y/n]y
                                       
